In [21]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import warnings
warnings.filterwarnings('ignore')

# Law data loading

- 사용자 - 아이템 - 평점이 기록된 데이터 가져오기
- pivot 을 통해 사용자-아이템 행렬로 변환(희소행렬)

In [23]:
import pandas as pd
import numpy as np

movies = pd.read_csv("/content/drive/MyDrive/DATA-33/1.ML/data/movies.csv")
ratings = pd.read_csv("/content/drive/MyDrive/DATA-33/1.ML/data/ratings.csv")

In [24]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [25]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


영화 정보와 평점 정보 합치기

In [26]:
rating_movies = pd.merge(ratings, movies, on='movieId')
rating_movies.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


# 사용자-영화 희소 행렬 만들기

In [27]:
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')
ratings_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# NaN값을 모두 0으로 변환 -> NaN 이 있으면 유사도를 구할 수가 없다.
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.5,3.5,0.0,0.0,0.0


# 영화-영화 유사도
사용자-영화 행렬을 전치하여 영화-사용자 행렬로 만들어 준 다음 유사도 구하기

In [29]:
ratings_matrix_T = ratings_matrix.T
ratings_matrix_T.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# 코사인 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# 유사도 행렬을 데이터 프레임으로 만들기
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)
item_sim_df.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Salem's Lot (2004),0.0,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Til There Was You (1997),0.0,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [31]:
# 'Godfather, The (1972)'와 비슷한 영화 찾기. TOP 10

item_sim_df['Godfather, The (1972)'].sort_values(ascending=False)[:10]

title
Godfather, The (1972)                                    1.000000
Godfather: Part II, The (1974)                           0.821773
Goodfellas (1990)                                        0.664841
One Flew Over the Cuckoo's Nest (1975)                   0.620536
Star Wars: Episode IV - A New Hope (1977)                0.595317
Fargo (1996)                                             0.588614
Star Wars: Episode V - The Empire Strikes Back (1980)    0.586030
Fight Club (1999)                                        0.581279
Reservoir Dogs (1992)                                    0.579059
Pulp Fiction (1994)                                      0.575270
Name: Godfather, The (1972), dtype: float64

In [32]:
item_sim_df['[REC] (2007)'].sort_values(ascending=False)[:10]

title
[REC] (2007)                               1.000000
Tremors 3: Back to Perfection (2001)       0.623737
Dog Soldiers (2002)                        0.621035
Wolf Creek (2005)                          0.601937
Joy Ride (2001)                            0.589097
Scooby-Doo 2: Monsters Unleashed (2004)    0.557882
[REC]³ 3 Génesis (2012)                    0.544207
Buried (2010)                              0.540871
Evil Dead (2013)                           0.537966
Dead Silence (2007)                        0.537966
Name: [REC] (2007), dtype: float64

# 평점 예측

- 아이템 기반 인접 이웃 협업 필터링으로 개인화된 영화 추천

$$
\hat{R}_{u, i}=\frac{\sum\big{(}S_{i,N} \cdot R_{u,N}\big{)}}{\sum\big{(}\big{|}S_{i,N}\big{|}\big{)}}
$$

In [35]:
def predict_rating(ratings_arr, item_sim_arr):
  ratings_pred = ratings_arr @ item_sim_arr / np.array([np.abs(item_sim_arr).sum(axis=1)])

  return ratings_pred

In [36]:
ratings_matrix.shape, item_sim.shape
# 유저/아이템 행렬, 아이템-아이템 유사도 행렬을 내적 -> 유저의 예측 평점이 나온다.

((610, 9719), (9719, 9719))

In [37]:
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)

ratings_pred_matrix.head()
# 예측점수 메트릭스

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501
4,0.049145,0.277628,0.160448,0.206892,0.309632,0.042337,0.130048,0.116442,0.099785,0.097432,...,0.051269,0.076051,0.055563,0.054137,0.008343,0.159242,0.100941,0.062253,0.146054,0.231187
5,0.007278,0.066951,0.041879,0.013880,0.024842,0.018240,0.026405,0.018673,0.021591,0.018841,...,0.009689,0.022246,0.013360,0.012378,0.000000,0.025839,0.023712,0.018012,0.028133,0.052315


# 예측 평가

가중치 평점 부여 뒤에 예측 성능 평가에 대한 MSE 구하기

1. 실제 평점 데이터 중 원래 값이 들어있던 위치를 구하기
2. 예측 평점 데이터 중 [1]에 있던 위치의 값과 MSE 계산

In [38]:
ratings_matrix.values.nonzero()
# 0이 아닌 값을 가진 좌표 윗 array가 행, 아래 array가 열

(array([  0,   0,   0, ..., 609, 609, 609]),
 array([  48,   66,  202, ..., 9712, 9715, 9716]))

In [41]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 구하기
def get_mse(pred, actual):
  # 실제 사용자가 평점을 부여한 위치의 데이터 (y)
  actual_y = actual[actual.nonzero()].flatten()

  # 실제 사용자가 평점을 부여한 위치의 예측 데이터 (pred)
  predict_y = pred[actual.nonzero()].flatten()

  print(actual_y)
  print(predict_y)


get_mse(ratings_pred, ratings_matrix.values)
# 전혀 유사하지 않은 정보도 포함해 예측 점수가 낮다.

[4.  4.  4.  ... 3.  2.  1.5]
[0.2855597  1.08359021 0.35404974 ... 2.57350896 1.08329872 1.81609065]


In [42]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 구하기
def get_mse(pred, actual):
  # 실제 사용자가 평점을 부여한 위치의 데이터 (y)
  actual_y = actual[actual.nonzero()].flatten()

  # 실제 사용자가 평점을 부여한 위치의 예측 데이터 (pred)
  predict_y = pred[actual.nonzero()].flatten()

  return mean_squared_error(predict_y, actual_y)

print("item기반 모든 인접 이웃 MSE : {}".format(get_mse(ratings_pred, ratings_matrix.values)))

item기반 모든 인접 이웃 MSE : 9.895354759094706


단순하게 유저-아이템 행렬과 아이템-아이템 유사도 행렬로 가중 평균 계산을 하면 유사하지 않은 아이템들도 포함되어 예측평점이 떨어진다.

In [47]:
# top-n 유사도를 가진 데이터들에 대해서만 예측 평점 계산

def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
  # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
  pred = np.zeros(ratings_arr.shape)

  # 사용자-아이템 평점  행렬의 열 크기만큼 반복 수행 (예측 평점을 하나씩 구하겠다는 의미)
  for col in range(ratings_arr.shape[1]):
    # 유사도 행렬에서 유사도가 큰 순으로 n 개 데이터 행렬의 index 반환, 인덱스만 가지고 온다
    top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1 :-1]] # col 번째 영화와의 유사도 추출

    # 개인화된 예측 평점을 계산(유저들이 영화에 부여한 평점 가지고오기)
    for row in range(ratings_arr.shape[0]):
      pred[row, col] = item_sim_arr[col, :][top_n_items] @ ratings_arr[row, :][top_n_items].T
                                     # similar array(행벡터)    @ 해당 유저가 부여한 상위 20개 영화에 대한 평점(행벡터)
      pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))

  return pred


In [48]:
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)

get_mse(ratings_pred, ratings_matrix.values)

3.6949827608772314

In [49]:
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.220798,0.0,0.0,1.677291,0.284372
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.220798,0.0,0.0,0.194828,0.000000
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


# 추천 시스템 작동

In [50]:
target_user_id = 78

In [51]:
# target_user_id 에 대한 모든 영화 정보
user_rating_id = ratings_matrix.loc[target_user_id, :]

# target_user_id가 평점을 부여한 영화들 확인
user_rating_id[user_rating_id >0].sort_values(ascending=False)[:10]

title
Die Hard (1988)                                                                   5.0
Airplane! (1980)                                                                  5.0
Terminator, The (1984)                                                            5.0
Terminator 2: Judgment Day (1991)                                                 4.5
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    4.5
Shawshank Redemption, The (1994)                                                  4.5
Ghostbusters (a.k.a. Ghost Busters) (1984)                                        4.5
Matrix, The (1999)                                                                4.5
Dodgeball: A True Underdog Story (2004)                                           4.5
Naked Gun: From the Files of Police Squad!, The (1988)                            4.5
Name: 78, dtype: float64

사용자가 관람하지 않은 영화 중에서 아이템 기반의 인접 이웃 협업 필터링 추천

In [53]:
def get_unseen_movies(ratings_matrix, userid):
  # UserId 로 입력받은 사용자의 모든 영화 정보를 추출하여 시리즈로 변환
  # 변환된 시리즈는 영화 이름(title)을 인덱스로 가지는 시리즈
  user_rating = ratings_matrix.loc[userid, : ]

  # user_rating이 0보다 크면 기존에 관람한 영화이다. 대상 인덱스를 추출하여 리스트 객체로 만들기
  already_seen = user_rating[user_rating >0].index.tolist()

  # 모든 영화명을 리스트로 만들기
  movies_list = ratings_matrix.columns.tolist()

  unseen_list = [movie for movie in movies_list if movie not in already_seen]

  return unseen_list

In [54]:
def recomm_movie_by_userid(pred_df, userid, unseen_list, top_n=10):
  recomm_movies = pred_df.loc[userid, unseen_list].sort_values(ascending=False)[:top_n]
  return recomm_movies

In [55]:
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, target_user_id)

# 추천 목록 만들기
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, target_user_id, unseen_list, top_n=10)

# 평점 데이터를 DataFrame으로
recomm_movies_df = pd.DataFrame(data = recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies_df

,pred_score
title,
Star Wars: Episode VI - Return of the Jedi (1983),2.226222
Braveheart (1995),1.985336
Star Wars: Episode V - The Empire Strikes Back (1980),1.929772
Seven (a.k.a. Se7en) (1995),1.780817
Schindler's List (1993),1.693000
Alien (1979),1.664949
Batman (1989),1.615901
Léon: The Professional (a.k.a. The Professional) (Léon) (1994),1.607490
Indiana Jones and the Temple of Doom (1984),1.552995
